<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Стабилизация перевернутого маятника на тележке

### Examples - Mechanics
<section class="post-meta">
By Eilif Sommer Øyre, Niels Henrik Aase and Jon Andreas Støvneng
</section>
Last edited: April 20th 2019

___

## Вступление
Перевернутый маятник - классический пример управляемости. Скорее всего, вы сами выполняли этот вид контроля, балансируя метлой или палкой на ладони. В этом случае маятник был физическим маятником, а ваша рука - внешней движущей силой. В этом блокноте мы попытаемся уравновесить перевернутый математический маятник на тележке, подверженной воздействию внешней силы, действующей на центр масс тележки.

<a id="img1" title="Oppenheim">
<img width=600 src="images/Oppenheim.png"></a>

*Рисунок 1: Профессор Массачусетского технологического института Алан В. Оппенгейм в своей лекции "Перевернутый маятник" из курса "Сигналы и системы". Источник: конспекты лекций Оппенгейма [[1]](#rsc)*.

На рисунке выше показана обычная экспериментальная установка для управления перевернутым маятником. Маятник перемещается, натягивая подвесной провод между тележкой и электрическим двигателем. В этой конкретной установке контроллер будет балансировать маятник независимо от массы "шарика" маятника, объясняя уверенность при наливании большего количества жидкости в стакан.

Первоначально мы смоделируем нашу маятниковую систему с использованием лагранжевой механики, а затем будем управлять маятником с помощью пропорционального управления, а затем построим алгоритм оптимизации регулирования на основе квадратичных весовых коэффициентов (линейный квадратичный регулятор [LQR])

In [ ]:
# pip install progressbar2

In [ ]:
pip install slycot   # optional
pip install control


In [ ]:
import numpy as np
import progressbar
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, Image
from scipy import signal
#import control
%matplotlib inline

## Теория Часть I

На рисунке 2 ниже показана схема системы, используемой в этом примере. Перевернутый маятник закреплен одним концом на верхней части тележки с массой $M$. Вращение вокруг неподвижной точки предполагается без трения. Шарик на конце маятника имеет массу $m$, в то время как жесткий стержень, соединяющий шарик с тележкой, не имеет массы. Это делает перевернутый маятник математическим маятником. Тележка может двигаться относительно поверхности, т. е. только в направлении $x$. Колеса включены только для иллюстрации. Сила, действующие на шарик, - это консервативная гравитационная сила $\vec{G}$ в отрицательном направлении $y$, в то время как сила, действующая на тележку, - это внешняя приложенная сила $\vec{u}$ в направлении $x$. Сила $\vec{F}$ является аппроксимацией трения во всей системе, также действующей в направлении $x$. Длина безмассового стержня назначается $l$, а угловое смещение маятника относительно $\vec{y}$ задается $\theta$.

<a id="img2" title="schematic of inverted pendulum on cart">
    
<img width=400 alt ="schematic"
src="images/drawing.png" ></a>

*Рисунок 2: Схема перевернутого маятника. Угол шарика $\theta$ равен нулю при неустойчивом (вертикальном) равновесии. Проиллюстрирована гравитационная сила $\vec{G}$, действующая на шарик с массой $m$, трение в системе $\vec{F}$ и внешняя сила $\vec{u}$.*

In [ ]:
figSizeX = 12
figSizeY = 9

# System parameters
m = 0.1          # mass of bead  [kg]
M = 1            # mass of cart  [kg]
l = 0.2          # length of rod [m]
g = 9.81         # gravitational acceleration [m s^-2]
mu = 10          # friction coefficient [kg s^-1]

# Initial conditions of generalised coordinates
x0_0 = 0         # cart position [m]
x1_0 = 0         # rod angle [radians]
x2_0 = 0         # cart velocity [m s^-1]
x3_0 = 0         # rod angular velocity [radians s^-1]
u_0 = 0          # initial cart force  [kg m s^-2]

Приведенные выше параметры системы могут быть изменены, чтобы получить различное движение и реакцию системы. Например, более массивный шарик потребовал бы большей внешней силы для балансировки, а более длинный стержень уменьшил бы период качания маятника.

### Уравнения движения

Можно вывести уравнения движения, используя ньютоновские силы, но лагранжева формулировка более элегантна. Полный вывод уравнений из лагранжиана 
$$ L = T - U $$ 
содержится в [приложении А](#apndxA).

Система, описанная на [рис. 2](#img2), имеет два голономных ограничения: Длина стержня $l$ фиксирована, а координата $y$ тележки всегда равна нулю. Это приводит к двум обобщенным координатам: угол $\theta$, положение тележки $x$ и их производные по времени $\dot{\theta}$ и $\dot{x}$. Соответствующие уравнения движения имеют вид

\begin{equation}
    \ddot{x} = \lambda(\theta)\big[mgl^2\dot{\theta}^2\sin\theta - mgl \sin\theta \cos\theta - l\mu \dot{x} + ul \big]
\label{cartacc}
\end{equation}

\begin{equation}
        \ddot{\theta} = \lambda(\theta)\big[(m + M)g \sin\theta - mgl\dot{\theta}^2 \sin\theta \cos\theta +  \mu\dot{x}\cos\theta - u\cos\theta \big]
\label{angelacc}
\end{equation}

где

\begin{equation}
    \lambda(\theta) = \frac{1}{l(M + m\sin^2\theta)}
\label{lambda} \text{.}
\end{equation}

Коэффициент трения $\mu$ исходит из силы трения $\vec{F}$, которая устанавливается пропорциональной скорости тележки, $F = -\mu\dot{x}$.

Переименовав обобщенные координаты, мы получим вектор состояния:

\begin{equation}
\textbf{x} =
\begin{bmatrix} x_0 \\
                x_1 \\
                x_2 \\
                x_3
\end{bmatrix} = 
\begin{bmatrix} x \\
                \theta \\
                \dot{x} \\
                \dot{\theta}
\end{bmatrix} \text{.}
\end{equation}

Затем, поняв, что $\dot{x_0} = x_2$ и $\dot{x_1} = x_3$, мы получим уравнение состояния:

\begin{equation}
    \dot{\textbf{x}} = \frac{d}{dt}
    \begin{bmatrix} x_0 \\
                    x_1 \\
                    x_2 \\
                    x_3
    \end{bmatrix} = 
    \begin{bmatrix} x_2 \\
                    x_3 \\
                    \lambda(x_1)\big[ml^2{x_3}^2\sin x_1 - mgl \sin x_1 \cos x_1 - l\mu x_2 + ul \big] \\
                    \lambda(x_1)\big[(m + M)g \sin x_1 - ml{x_3}^2 \sin x_1 \cos x_1 +  \mu x_2\cos x_1 - u\cos x_1 \big]
    \end{bmatrix} \text{.}
\label{stateEquation}
\end{equation}

Последние два уравнения нелинейны. Это означает, что движение системы очень трудно регулировать. Следовательно, нам придется их линеаризовать. Это делается в части II.

## Реализация - Часть I
### Моделирование системы

Чтобы смоделировать перевернутый маятник на тележке, мы должны итеративно решить уравнение состояния. В этом блокноте мы будем использовать численный метод *Рунге-Кутта четвертого порядка* (RK4) для интегрирования системы нелинейных дифференциальных уравнений. Если вы никогда не слышали об этом методе или хотите получить краткое описание, ознакомьтесь с [этим блокнотом](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/runge_kutta_method.ipynb).

В дополнение к определению функции шага RK4 мы определяем функцию, которая возвращает правую часть системы с учетом вектора $\textbf{x}$ и параметров системы, а также функцию для вычисления потенциала и кинетической энергии системы.

In [ ]:
def RHS(z, RHSargs): 
        """Given a specified state vector and system parameters,
        z and RHSargs respectively, this function returns the 
        state vector element's time derivative, i.e. the right 
        hand side of the state equations.
        Parameters:
            z        4x1 array
            RHSargs  6x1 array
        Returns:
            dxdt     4x1 array
        """
        m, M, l, g, mu, u = RHSargs
        x0, x1, x2, x3 = z
        dx0dt = x2
        dx1dt = x3
        lambdax1 = 1/(l*(M + m*np.sin(x1)**2))
        
        dx2dt = lambdax1*(m*l*l*x3**2*np.sin(x1) - m*g*l*np.sin(x1)*np.cos(x1)
                            - l*mu*x2  + u*l)
        dx3dt = lambdax1*((m + M)*g*np.sin(x1) - m*l*x3*x3*np.sin(x1)*np.cos(x1)
                            + mu*x2*np.cos(x1) - u*np.cos(x1))
        
        dxdt = np.array([dx0dt, dx1dt, dx2dt, dx3dt])
                               
        return dxdt
                               

def RK4Step(z, dt, RHS, RHSargs):
        """ Performs one step of the 4th order Runge-Kutta method
        z is the current state vector, dt is the numerical time
        step, RHS is a function for the right hand side of the
        system of differential equations, and RHSargs is the input
        parameters for that function. RK4step returns the next
        state vector, i.e. the values of z after a time dt.
        Parameters:
            z         4x1 array
            dt        float
            RHS       function
            RHSargs   6x1 array
        Returns:
            The state vactor, z, after a time, dt.
        """
        k1 = np.array(RHS(z, RHSargs))
        k2 = np.array(RHS(z + k1*dt/2, RHSargs))
        k3 = np.array(RHS(z + k2*dt/2, RHSargs))
        k4 = np.array(RHS(z + k3*dt, RHSargs))
        
        return z + dt/6*(k1 + 2*k2 + 2*k3 + k4)

In [ ]:
def addEnergy(z):
    """ Given the the state vector, z, calculates and returns the kinetic
    and potential energy of the system in an arrray. See appendix B for the
    energy term formulas.
    """
    x0, x1, x2, x3 = z
    U = m*g*l*np.cos(x1)
    T = 0.5*x2**2*(m + M) + 0.5*m*x3**2*l**2 + m*x2*x3*l*np.cos(x1)
    return np.array([T, U])

Теперь мы можем интегрировать уравнение состояния после определения размера шага, $\Delta t$ и продолжительности интегрирования $t_{max}$. Матрица `Z` инициализируется, чтобы содержать вектор состояния в любое время $t$. В то время как начальный угол маятника равен нулю, он никогда не упадет, потому что x-составляющая гравитационной силы абсолютно равна нулю. Таким образом, чтобы сделать ситуацию более реалистичной, в качестве внешней силы добавляется случайный шум, $u$, в каждой итерации. Шум может соответствовать, например, порыву ветра.

In [ ]:
duration = 10                # Integration duration [sec]
timeStep = 1e-2              # Time step 
n = int(duration/timeStep)   # Number of datapoints

# Initialising matrices
RHSargs = np.array([m, M, l, g, mu, u_0]) # System parameters
Z = np.zeros((4, n))                      # Z = [[x0_0, ... x0_n], [x1_0, ... x1_n], [x2,...,], [x3,...,]
# Inserting initial state vector into Z
Z[:, 0] = np.array([x0_0, x1_0, x2_0, x3_0])
# Initialising matrix containing the noise and potential force on the cart
extForce = np.zeros((2, n))
extForce[0, 0] = u_0

# Initialising matrix containing the kinetic and potential energy of the system
energy = np.zeros((2, n))    # energy = [[T_0 ... T_n], [U_0, ... U_n]]
energy[:, 0] = addEnergy([x0_0, x1_0, x2_0, x3_0])

bar = progressbar.ProgressBar()
for i in bar(range(n - 1)):
    # Compute and insert the next state vector using RK4
    Z[:, i + 1] = RK4Step(Z[:, i], timeStep, RHS, RHSargs)
    # Generating a random noise value
    noise = np.random.uniform(-0.001, 0.001)
    extForce[1, i +1] = noise
    # Updating the system parameter u
    RHSargs[5] = noise
    # Add the energy terms
    energy[:, i + 1] = addEnergy(Z[:, i + 1])

Затем мы составляем график фазового пространства обобщенных координат в дополнение к эволюции вектора состояния. Определение функции построения графика приведено в [Приложении B](#apndxB).

In [ ]:
plotData(Z, extForce, energy, appliedForce=False)

График фазового пространства $\theta$ показывает движение, подобное простому маятнику с трением. Он вращается по спирали в направлении нулевой угловой скорости и угла $\theta = \pi$. Эта точка будет соответствовать маятнику, висящему вниз в состоянии покоя, что является интуитивным результатом, если никакая сила не пытается сохранить перевернутую форму. Два самых нижних графика показывают, как общая энергия системы уменьшается со временем. Это происходит из-за силы трения ($\vec{F}$ в [рис. 2](#img2)), действующей в направлении, противоположном скорости тележки. Чем больше коэффициент трения $\mu$, тем выше потери энергии. Как видно из графиков эволюции времени, маятник остается перевернутым некоторое время, прежде чем шум опрокинет его. Однако путь в фазовом пространстве для $q_2(x, \dot{x})$ имеет не столь гармоническое движение. Кажется, что он испытывает высокое ускорение в течение короткого времени два раза при каждом периодическом движении. Давайте сделаем анимацию, чтобы лучше видеть, что происходит. Определение функции анимации приведено в [Приложение C](#apndxC). Приведенная ниже анимация была сгенерирована с использованием кода:

    generateAnimationWithPhaseSpace(Z, 'falling_phsp.gif')
    
![falling](images/falling_phsp-kopi2.gif)

Как видно из анимации, тележка скользит взад и вперед из-за маятника.

Теперь в нашем уравнении состояния (уравнения движения) есть переменная $u$, которая позволяет нам оказывать внешнюю силу на тележку. В следующей части блокнота мы будем использовать эту переменную, чтобы попытаться стабилизировать маятник. Но сначала уравнение состояния должно быть линеаризовано.

## Теория Часть II
### Линеаризация около равновесия
Нелинейные уравнения движения могут быть линеаризованы в виде

\begin{equation}
   \dot{\textbf{x}}
    = \mathcal{A}\textbf{x}  + \mathcal{B}u
\end{equation}

около положения равновесия. Перевернутый маятник в нашем примере имеет две точки равновесия. Устойчивое равновесие, соответствующее висящему вниз маятнику

\begin{equation}
    \begin{cases} x_0 &= \mathrm{free} \\
                   x_1 &= \pi  \\
                   x_2 &= 0 \\
                   x_3 &= 0
    \end{cases} . 
\end{equation}

И неустойчивое равновесие, соответствующее маятнику, тщательно сбалансированному вертикально. Он нестабилен в том смысле, что любое возмущение, например шум в виде небольшого порыва ветра, заставит его упасть.

\begin{equation}
    \begin{cases} x_0 &= \mathrm{free} \\
                   x_1 &= 0  \\
                   x_2 &= 0 \\
                   x_3 &= 0
    \end{cases} . 
\end{equation}

Последнее и есть то равновесие, которое нас интересует.

Матрицы $\mathcal{A}$ и $\mathcal{B}$ задаются матрицей Якоби,

\begin{equation}
\mathcal{J} = \big [ \frac{\partial \textbf{f}}{\partial x_1} \cdots \frac{\partial \textbf{f}}{\partial x_n} \big ] = 
    \begin{bmatrix} \frac{\partial f_0}{\partial x_0} & \cdots & \frac{\partial f_0}{\partial x_n} \\
                    \vdots & \ddots & \vdots \\
                    \frac{\partial f_n}{\partial x_0} & \cdots & \frac{\partial f_n}{\partial x_n}
    \end{bmatrix}
\end{equation}

[[2]](#rsc). Затем производные оцениваются в точке равновесия. Здесь $f_i$ - компоненты уравнения состояния

\begin{equation}
\dot{\textbf{x}} = \frac{d}{dt}
    \begin{bmatrix} x_0 \\
                    x_1 \\
                    x_2 \\
                    x_3
    \end{bmatrix} =
    \begin{bmatrix} f_0 \\
                    f_1 \\
                    f_2 \\
                    f_3
    \end{bmatrix} = 
    \begin{bmatrix} x_2 \\
                    x_3 \\
                    \lambda(x_1)\big[ml^2{x_3}^2\sin x_1 - mgl \sin x_1 \cos x_1 - l\mu x_2 + ul \big] \\
                    \lambda(x_1)\big[(m + M)g \sin x_1 - ml{x_3}^2 \sin x_1 \cos x_1 +  \mu x_2\cos x_1 - u\cos x_1 \big]
    \end{bmatrix} \text{.}
\end{equation}

При вычислении матрицы Якоби в неустойчивом равновесии каждая координата, кроме $x_0$, после дифференцирования обращается в ноль. Это дает

\begin{equation}
\mathcal{A} = 
    \begin{bmatrix} \frac{\partial f_0}{\partial x_0} \big |_{0} & \frac{\partial f_0}{\partial x_1} \big |_{0} & \frac{\partial f_0}{\partial x_2} \big |_{0} & \frac{\partial f_0}{\partial x_3} \big |_{0} \\
                    \frac{\partial f_1}{\partial x_0} \big |_{0} & \frac{\partial f_1}{\partial x_1} \big |_{0} & \frac{\partial f_1}{\partial x_2} \big |_{0} & \frac{\partial f_1}{\partial x_3} \big |_{0} \\
                    \frac{\partial f_2}{\partial x_0} \big |_{0} & \frac{\partial f_2}{\partial x_1} \big |_{0} & \frac{\partial f_2}{\partial x_2} \big |_{0} & \frac{\partial f_3}{\partial x_3} \big |_{0} \\
                    \frac{\partial f_3}{\partial x_0} \big |_{0} & \frac{\partial f_3}{\partial x_1} \big |_{0} & \frac{\partial f_3}{\partial x_2} \big |_{0} & \frac{\partial f_3}{\partial x_3} \big |_{0} 
    \end{bmatrix} = 
    \begin{bmatrix} 0 & 0 & 1 & 0 \\
                    0 & 0 & 0 & 1 \\
                    0 & \frac{-mg}{M} & \frac{-\mu}{M} & 0 \\
                    0 & \frac{g(m + M)}{lM} & \frac{\mu}{ML} & 0
    \end{bmatrix}
\end{equation}

и

\begin{equation}
\mathcal{B} = 
    \begin{bmatrix} \frac{\partial f_0}{\partial u} \big |_{0} \\
                    \frac{\partial f_1}{\partial u} \big |_{0} \\
                    \frac{\partial f_2}{\partial u} \big |_{0} \\
                    \frac{\partial f_3}{\partial u} \big |_{0}
    \end{bmatrix} = 
    \begin{bmatrix} 0  \\
                    0  \\
                    \frac{1}{M} \\
                    \frac{-1}{Ml}
    \end{bmatrix} \text{.}
\end{equation}

Можно было бы провести тест, чтобы показать, что система управляется с помощью матриц $\mathcal{A}$ и $\mathcal{B}$, но в нашем случае мы будем считать, что это так.

Теперь, определив контроллер

$$ u = -\mathcal{K}\textbf{x}, $$

мы получаем линеаризованную форму 

\begin{equation}
\dot{\textbf{x}} = (\mathcal{A} - \mathcal{BK})\textbf{x},
\label{controller}
\end{equation}

где $\mathcal{K}$ - матрица усиления. Это *пропорциональное* усиление, поскольку управляющая переменная (внешняя сила $u$) *пропорциональна* вектору состояния $\text{bf}$. Это позволяет нам выбрать $\mathcal{K}$ таким образом, чтобы собственные значения матрицы в скобках были стабильными, по существу отрицательными (что указывает на отрицательную обратную связь) [[1, 3]](#рск).

## Реализация Часть II
### Пропорциональный регулятор
В этой части мы выполняем аналогичную реализацию, как описано выше, в дополнение к вычислению матрицы усиления. Затем, добавляя управляющую силу на каждом шаге интегрирования, создаем петлю обратной связи. Чтобы получить матрицу $\mathcal{K}$ такую, чтобы собственные значения $(\mathcal{A} - \mathcal{BK})$ стали искомыми собственными значениями используется `poles` из scipy [`signal.place_poles`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.place_poles.html). Этот пакет просто возвращает матрицу усиления с учетом полюсов, $\mathcal{A}$ и $\mathcal{B}$.

In [ ]:
# Redefining the initial rod angle in such a way 
# that it starts out by falling
x1_0 = 0.2

# Defining the setpoints. I.e. the values of the 
# state vector needed for stabilasion
# x0 can be choosed freely, we choose -0.2
setpoints = [-0.2,   
                0,
                0,
                0]

# Defining the control matrices A and B
A = np.array([[0, 0, 1, 0],
            [0, 0, 0, 1],
            [0, -m*g/M, -mu/M, 0],
            [0, (m+M)*g/(M*l), mu/(M*l), 0]])
                    
B = np.array([[0], 
            [0], 
            [1/M],
            [-1/(M*l)]])

# Specifying wanted eigenvalues of the matrix (A - B*K)
# Subscript PG stands for Proportional Gain
poles_PG = [-1.3, -1.4, -1.5, -1.6]
# Get the full state feedback bunch object
fsf = signal.place_poles(A, B, poles_PG)
# Extract the gain matrix
K_PG = fsf.gain_matrix

def controller(z, K):
    """ Given the current values of the state vactor, z
    and the gain matrix K, it returns the current control
    input, u.
    """
    offset = z - setpoints
    u = -np.dot(K, offset)
    return u[0]


def integrateControlled(K, pushForce, pushTime):
    """ This is the integration process implemented earlier,
    but with controll input added to the force. The possibility
    for a 'push' on the pendulum is also added.
    """
    # Re-initialising matrices
    RHSargs = np.array([m, M, l, g, mu, u_0])
    # Z = [(x0_0, x0_1, ..., x0_n), (x1_0, x1_1, ..., x1_n), (x2, ...,), (x3, ...,)]
    Z = np.zeros((4, n))
    Z[:, 0] = np.array([x0_0, x1_0, x2_0, x3_0])
    # Array containing the external forces at each time step
    extForce = np.zeros((2, n))
    extForce[0, 0] = u_0

    # Initialising matrix containing the kinetic and potential energy of the system
    energy = np.zeros((2, n))    # energy = [[T_0 ... T_n], [U_0, ... U_n]]
    energy[:, 0] = addEnergy([x0_0, x1_0, x2_0, x3_0])

    bar = progressbar.ProgressBar()
    for i in bar(range(n-1)):
        Z[:, i + 1] = RK4Step(Z[:, i], timeStep, RHS, RHSargs)
        
        # "Push"
        #  Non-continuously increasing/reducing the angular velocity
        if i == pushTime and i>0:
            Z[3, i + 1] += pushForce
            print("Pushed!")
        
        noise = np.random.uniform(-0.01, 0.01)
        extForce[1, i +1] = noise
        # Calculate the control variable
        force = controller(Z[:, i + 1], K)
        # Store the force, u
        extForce[0, i + 1] = force
        # Adding the total force and updating the parameters
        f = noise + force
        RHSargs[5] = f
        # Adding energy terms
        energy[:, i + 1] = addEnergy(Z[:, i + 1])
        
    return Z, extForce, energy

Z_PG, extForce_PG, energy_PG = integrateControlled(K_PG, 0, 0)

In [ ]:
plotData(Z_PG, extForce_PG, energy_PG, appliedForce=True)

И маятник уравновешен! Элементы вектора состояния колеблются вокруг заданных значений перед стабилизацией. Начальные точки находятся на положительных наклонах в фазовом пространстве, что указывает на нестабильное начальное положение. В конечной точке наклон отрицательный (при $\theta = 0$), что становится возможным благодаря управляющей силе.

### Линейный квадратичный Регулятор (LQR)
Теперь мы смогли стабилизировать маятник на его вертикальном равновесии, но при этом мы выбираем некоторые случайные отрицательные собственные значения (полюса) $(\mathcal{A} - \mathcal{BK})$. Собственные значения могут быть скорректированы еще более отрицательно, что приводит к более мощному регулированию. Однако в реальной инженерной ситуации будет конечная максимальная приложимая сила $u$ и другие физические ограничения, и может потребоваться, чтобы некоторые координаты стабилизировались быстрее, чем другие. Таким образом, было бы привлекательно оптимизировать контроль. *Функция затрат* имеет вид [[4]](#rsc)

\begin{equation}
J(u) = \int^{\infty}_0 \big(\textbf{x}^{T}\mathcal{Q}\textbf{x} + u^{T}Ru)dt
\label{CostFunction}
\end{equation}

Она представляет собой сумму отклонений элементов вектора состояния от заданных точек и приложенной силы. Минимизируя эту функцию с помощью весовых коэффициентов ($\mathcal{Q}$ и $R$), можно найти полюса, которые минимизируют нежелательные отклонения. Значение функции можно рассматривать как штраф. Если система не стабилизируется быстро, первый штраф будет большим. Если контроллер тратит много энергии на стабилизацию, второй штраф будет большим. Скажем, например нам не нравится иметь высокую угловую скорость в нашей системе, и у нас есть мощный двигатель с дешевым электричеством, весовые коэффициенты могут быть определены

\begin{equation}
\mathcal{Q} = 
    \begin{bmatrix} 1 & 0 & 0 & 0 \\
                    0 & 1 & 0 & 0 \\
                    0 & 0 & 1 & 0 \\
                    0 & 0 & 0 & 10
    \end{bmatrix} , \qquad R = 0.1
\end{equation}

Пакет python `control` содержит функцию [`lqr`](https://python-control.readthedocs.io/en/0.8.2/generated/control.lqr.html), который оптимизирует функцию затрат для нас, и просто возвращает матрицу выигрыша $\mathcal{K}$, соответствующую нашим весовым коэффициентам. Он имеет ту же функциональность, что и функция MATLAB `lqr` [[5]](#rsc).

In [ ]:
# Weighing factor for the state vector
Q = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 10]])
# Weighing factor for the external force
R = 0.1

# Find the gain matrix and poles using lqr-function
K_LQR, s, poles_LQR = control.lqr(A, B, Q, R)

# Integrate and retrieve the data
Z_LQR, extForce_LQR, energy_LQR = integrateControlled(K_LQR, 0, 0)
plotData(Z_LQR, extForce_LQR, energy_LQR, appliedForce=True)

Код

    generateAnimation(Z_PG, Z_LQR, "lqr_controlled.gif")

генерирует анимацию ниже. См. [приложение C](#apndxC) для определения функции.

![controlled](images/lqr_controlled-kopi.gif)

Из графиков и амимации выше мы видим значительное улучшение времени стабилизации. Угловая скорость достигает заданной точки через две секунды, в то время как контроллер тратит больше времени на стабилизацию положения и скорости тележки, отражая коэффициенты взвешивания. Кроме того, в системе с LQR гораздо меньше движения. Это прямой результат "наказания" отклонений.

Давайте дадим маятнику небольшой "толчок" через 4 секунды и посмотрим, что произойдет. С шагом по времени 0.01 секунды толчок приурочен к итерации номер 400.

In [ ]:
Z_LQR_pushed, extForce_LQR_pushed, energy_LQR_pushed = integrateControlled(K_LQR, .5, 400)
plotData(Z_LQR_pushed, extForce_LQR_pushed, energy_LQR_pushed, appliedForce=True)

Мы видим, что маятник быстро стабилизируется после толчка, который резко (прерывисто) изменяет наклон траектории фазового пространства.

___

<a id="rsc"></a>
## Resources and Further Reading
<a>[1]</a> Prof. Alan V. Oppenheim, Demonstration from lecture notes 26, *Feedback example: The Inverted Pendulum*, from the MIT course *Signals and Systems*. Assessed January 31st 2019. [Click here for Lecture notes](https://ocw.mit.edu/resources/res-6-007-signals-and-systems-spring-2011/lecture-notes/MITRES_6_007S11_lec26.pdf). [Click here for video lecture](https://www.youtube.com/watch?v=D3bblng-Kcc).

<a>[2]</a> Packard, Polo, Horowitz, 2002, [*Jacobien Linearizations, equilibrium points*](https://www.cds.caltech.edu/~murray/courses/cds101/fa02/caltech/pph02-ch19-23.pdf), from Dynamic Systems and feedback. Assessed March 8th.

<a>[3]</a> Davis, [*9.3. Equilibrium: Stable or Unstable?*](https://web.ma.utexas.edu/users/davis/375/popecol/lec9/equilib.html), Department of mathematics, University of Texas at Austin. Assessed March 8th.

<a>[4]</a> [Wikipedia](https://en.wikipedia.org/wiki/Linear%E2%80%93quadratic_regulator) has a nice description of LQR and briefly illuminates its application potentials.

<a>[5]</a> MathWorks, [documentation of the function `lqr`](https://www.mathworks.com/help/control/ref/lqr.html).

<a>[6]</a> Prof. Jerrold D. Marsden, [*1 Lecture: Lagrangian Mechanics*](http://www.cds.caltech.edu/archive/help/uploads/wiki/files/212/Mechanics_140a_old.pdf), Caltech, November 21st 2006. Assessed February 4th 2019.

___

<a id="apndxA"></a>
## Приложение А: Вывод уравнений движения

Следующие уравнения движения соответствуют системе в [рис. 2](#img2). Во-первых, найдено выражение для кинетической энергии и потенциальной энергии, $T$ и $U$, системы. Затем из лагранжиана $L = T - U$ уравнения движения вычисляются с использованием уравнения Эйлера-Лагранжа

\begin{equation}
\frac{d}{dt} \frac{\partial L}{\partial \dot{q}_i} - \frac{\partial L}{\partial q_i} = 0 \text{.}
\label{EL} \quad(1)
\end{equation}

Поскольку стержень не имеет массы, кинетическая энергия системы равна сумме кинетической энергии тележки и шарика,

$$ T = \frac{1}{2}M v^{2}_c + \frac{1}{2}m v^{2}_b .$$

Тележка неподвижна по оси y,

$$ v^2_c = \dot{x}_c^2 + \dot{y}_c^2 = \dot{x}_c^2 .$$

Используя угол наклона шарика, $\theta$ относительно оси y,

\begin{equation}
    \begin{aligned}
        x_b &= x_c + l \sin \theta \\
        y_b &= l\cos \theta \\
        \rightarrow \dot{x}_b &= \dot{x}_c  +\dot{\theta}l \cos \theta \\
        \dot{y}_b &= -\dot{\theta} l \sin \theta \\
        \rightarrow v^2_b &= \dot{x}^2_c + \dot{\theta}^2 l^2 + 2\dot{x}_c \dot{\theta}l \cos \theta
    \end{aligned}
\end{equation}

Таким образом, с потенциальной энергией

$$ U = mgl\cos \theta ,$$

и переписывая $x_c \rightarrow x$, лагранжиан становится

\begin{equation}
L = \frac{1}{2} \dot{x}^2(m + M) + \frac{1}{2}m\dot{\theta}^2l^2 + m\dot{x}\dot{\theta}l\cos\theta - mgl\cos\theta
\label{L} \quad(2)
\end{equation}

Применяя (1) к обобщенным координатам $q_i = \theta$, мы получаем

\begin{equation}
\ddot{\theta} = \frac{g}{l}\sin\theta - \frac{\ddot{x}}{l}\cos\theta .
\label{I} \quad(3)
\end{equation}

Вторая обобщенная координата $q_i = x$ участвует во внешней силе $u$ и неконсервативном силовом трении. Таким образом, уравнение Эйлера-Лагранжа заменяется принципом Лагранжа-д'Аламбера, который эквивалентен уравнениям Эйлера-Лагранжа с внешней силой [[6]](#rsc):

\begin{equation}
\frac{d}{dt} \frac{\partial L}{\partial \dot{q}_i} - \frac{\partial L}{\partial q_i} = F_{ext}(q, \dot{q}) \text{.}
\label{ELEF} \quad(4)
\end{equation}

Это дает уравнение движения

\begin{equation}
\ddot{x} = \frac{1}{m+M}\big [ ml\big(\dot{\theta}^2\sin\theta - \ddot{\theta}\cos\theta\big) - \mu\dot{x} + u \big ],
\label{II} \quad(5)
\end{equation}

где $-\mu\dot{x}$ - аппроксимация силы трения в системе ($\mu$ - коэффициент трения), а $u$ - внешняя сила, приложенная к тележке.

В матричной форме (3) и (5) можно записать

\begin{equation}
    \begin{bmatrix} m+M & ml\cos\theta \\
                    \cos\theta & l 
    \end{bmatrix}
    \begin{bmatrix} \ddot{x} \\
                    \ddot{\theta}
    \end{bmatrix} = 
    \begin{bmatrix} ml\dot{\theta}^2\sin\theta - \mu\dot{x} \\
                    g\sin\theta
    \end{bmatrix} + 
    \begin{bmatrix} 1 \\
                    0
    \end{bmatrix} u .
\end{equation}

Наконец, определив 

\begin{equation}
    \Lambda(\theta) =
    \begin{bmatrix} m+M & ml\cos\theta \\
                    \cos\theta & l 
    \end{bmatrix}
\end{equation}

полученные уравнения движения становятся

\begin{equation}
    \begin{bmatrix} \ddot{x} \\
                    \ddot{\theta}
    \end{bmatrix} = \Lambda(\theta)^{-1}
    \begin{bmatrix} ml\dot{\theta}^2\sin\theta - \mu\dot{x} \\
                    g\sin\theta
    \end{bmatrix} + 
    \begin{bmatrix} 1 \\
                    0
    \end{bmatrix} u ,
\label{EQM}
\end{equation}

где $\Lambda(\theta)^{-1}$ - обратная величина $\Lambda(\theta)$,

\begin{equation}
\Lambda(\theta)^{-1} = \frac{1}{l(M+m\sin^2\theta)}
    \begin{bmatrix} l & -ml\cos\theta \\
                    -\cos\theta & m+M
    \end{bmatrix}.
\end{equation}


<a id="apndxB"></a>
## Appendix B - Definition of `plotData`

In [ ]:
def relative(data):
    """Returns the values of 'data' relative to the absolute maximum value of 'data'."""
    return data/np.amax(np.absolute(data))
        
        
def angle(data):
    """Maps the values of data on to the the interval [1, -1]"""
    copy = np.zeros(data.shape)
    for i in range(len(data)):
        if data[i] <= np.pi:
            copy[i] = (np.pi - data[i])/np.pi
        else:
            copy[i] = -(data[i] - np.pi)/np.pi
    return copy

In [ ]:
def plotData(Z, extForce, energy, appliedForce):
        """ This function plots the results in time-evolution, and phase-space plots.
        It differ between two functionalities, given by the boolean 'appliedForce'.
        Arguments:
                Z              4xN array. The state vector component-values at each timestep
                extForce       2xN array. The value of external force and noise at each timestep
                energy         2xN array. The kinetic and potential energy at each timestep
                appliedForce   boolean.
        """
        # Defining an array of the time in seconds at each step
        time = np.arange(0, n*timeStep, timeStep)
        totalF = extForce[0] + extForce[1]
        
        # Plots time evolution of the external force and every state-vector-component if true
        if appliedForce:
            plt.figure(figsize=(12, 6), dpi=200)
            plt.subplot(2, 2, (1, 3))
            plt.title("Time evolution of general coordinates")
            plt.plot(time, np.zeros(n), 'b--',label='Setpoint', linewidth=1)
            plt.plot(time, [setpoints[0]/np.amax(np.absolute(Z[0])) for i in range(n)], 
                     '--', color="black", linewidth=1, label=r"Setpoint for $x$")
            plt.plot(time, relative(Z[0]), label=r'$x$')
            plt.plot(time, relative(Z[1]), label=r"$\theta$")
            plt.plot(time, relative(Z[2]), label=r"$\dot{x}$")
            plt.plot(time, relative(Z[3]), label=r"$\dot{\theta}$")
            plt.plot(time, relative(extForce[0]), label=r"$u$")
            plt.xlabel('time, sec')
            plt.legend(loc='best')
            plt.grid(linestyle=':')
        # Plots time evolution of the cart position and rod angle only if appliedForce == false.
        else:
            plt.figure(figsize=(12, 6), dpi=200)
            plt.subplot(2, 2, (1, 3))
            plt.title("Time evolution of general coordinates")
            plt.plot(time, relative(Z[0]), label=r'$x$')
            plt.plot(time, angle(Z[1]), label=r"$\theta$")
            plt.xlabel('time, A.U')
            plt.legend(loc='best')
            plt.grid(linestyle=':')
        
        # Independent of appliedForce.
        # Generate phase-space plot of the coordinate theta.
        plt.subplot(222)
        plt.title("Rod angle phase-space plot")
        plt.plot(Z[1], Z[3])
        plt.plot(Z[1][-1], Z[3][-1], 'o', label="Final point")
        plt.plot(Z[1][0], Z[3][0], '*', label="Initial point")
        plt.xlabel(r'$\theta$ [rad]')
        plt.ylabel(r'$\dot{\theta}$ [rad/s]')
        plt.legend(loc='best')
        plt.grid(linestyle=":")
        
        # Generates phase-space plot of the coordinate x.
        plt.subplot(224)
        plt.title("Cart position phase-space plot")
        plt.plot(Z[0], Z[2])
        plt.plot(Z[0][-1], Z[2][-1], 'o', label="Final point")
        plt.plot(Z[0][0], Z[2][0], '*', label="Initial point")
        plt.xlabel(r'$x$ [m]')
        plt.ylabel(r'$\dot{x}$ [m/s]')
        plt.legend(loc='best')
        plt.grid(linestyle=":")
        
        # Adjusts horisontal seperation between subplots.
        plt.subplots_adjust(hspace=0.4)
        
        if appliedForce == False:
            # Plots the loss of total energy as a function of time
            plt.figure(figsize=(12, 6), dpi=200)
            plt.subplot(121)
            plt.title("Loss of total energy due to friction")
            # Minimum total energy of the system
            minima = m*g*l*np.cos(np.pi)
            # The y-axis displays enrgy percentage relative to initial total energy
            plt.plot(time, (np.sum(energy, 0) - minima)/(np.sum(energy[:,0], 0) - minima)*100)
            plt.xlabel("time, sec")
            plt.ylabel('Percentage of initial total energy, %')
            plt.grid(linestyle=":")
            
            # Plots the kinetic and potential energy of the system as a function of time.
            plt.subplot(122)
            plt.title("Kinetic and potential energy of the system")
            plt.xlabel("time, sec")
            plt.ylabel("energy, J")
            plt.plot(time, energy[0], label="T")
            plt.plot(time, energy[1], label="U")
            plt.legend(loc='best')
            plt.grid(linestyle=":")

        plt.show()

<a id="apndxC"></a>
## Appendix C - Definition of `generateAnimation` and 
## `generateAnimationWithPhaseSpace`

In [ ]:
def cartX(x_1):
    """ Given the value of x_1, i.e. the position
    of the cart, returns the x-coordinates of the
    cart's corners."""
    return np.array([x_1 - l/8, x_1 + l/8, x_1 + l/8, x_1 - l/8, x_1 - l/8])

In [ ]:
def generateAnimation(Z, Z2, name):
    """ This function generates two animations of the system as a function of time,
    from the two results, 'Z' and 'Z2'. The two animations are plottet underneath each other.
    Arguments:
            Z      4xN array. State vector component-values at each timestep
            Z2     4xN array. State vector component-values at each timestep
            name   string. Name of the animation file.
    """
    
    # Defining an array of the time in seconds at each step
    timeValues = np.arange(0, n*timeStep, timeStep)
    
    X = Z[0] + l*np.sin(Z[1])                           # x-coordinates of bead
    Y = l*np.cos(Z[1])                                  # y-coordinates of bead
    cartY = np.array([l/16, l/16, -l/16, -l/16, l/16])  # y-coordinates of cart corners
    
    X2 = Z2[0] + l*np.sin(Z2[1])                           # x-coordinates of bead 2
    Y2 = l*np.cos(Z2[1])                                   # y-coordinates of bead 2
    cartY2 = np.array([l/16, l/16, -l/16, -l/16, l/16])    # y-coordinates of cart corners 2
    
    fig = plt.figure(figsize=(figSizeX, figSizeY), dpi=200)
    
    # Animation/result/pendulum number one
    plt.subplot(211)
    # Axis limitations
    xMin = np.min(X) - 1.5*l
    xMax = np.max(X) + 1.5*l
    # Adjust the y-limitations such that a circle looks like a circle.
    # i.e. no scaling.
    xDomain = xMax - xMin
    yDomain = xDomain * 0.5*figSizeY/figSizeX
    plt.xlim(xMin, xMax)
    plt.ylim(-0.5*yDomain, 0.5*yDomain)

    # Defining the different elements in the animation
    surface, = plt.plot([xMin, xMax], [-l/16, -l/16], color='black', linewidth=1)    # The surface
    tail, = plt.plot(X[0], Y[0], '--', color="blue")            # Previous position of the pendulum bead
    cart, = plt.plot(cartX(Z[0, 0]), cartY, color="red")        # The cart 
    rod, = plt.plot([Z[0, 0], X[0]], [0, Y[0]], color="black")  # The massless pendulum rod of length l
    bead, = plt.plot(X[0], Y[0], 'o', color="black", ms=4)      # The pendulum bead
    text = plt.text(xMax-0.1, 0.5*yDomain - 0.1, r'$t =  %.2f$s'%(timeValues[0]), # Text box with elapsed time
            {'color': 'k', 'fontsize': 10, 'ha': 'center', 'va': 'center',
            'bbox': dict(boxstyle='round', fc='w', ec='k', pad=0.2)})

    plt.xlabel('x, m')
    plt.ylabel('y, m')
    plt.title("Proportional Gain controller")
    
    # Animation/results/pendulum number two
    plt.subplot(212)
    # Axis limitations
    xMin2 = np.min(X2) - 3*l 
    xMax2 = np.max(X2) + 3*l 
    xDomain2 = xMax2 - xMin2
    yDomain2 = xDomain2 * 0.5*figSizeY/figSizeX
    plt.xlim(xMin2, xMax2)
    plt.ylim(-0.5*yDomain2, 0.5*yDomain2)

    # Defining the different elements in the animation
    surface2, = plt.plot([xMin2, xMax2], [-l/16, -l/16], color='black', linewidth=1)    # The surface
    tail2, = plt.plot(X2[0], Y2[0], '--', color="blue")              # Previous position of the pendulum bead
    cart2, = plt.plot(cartX(Z2[0, 0]), cartY2, color="red")         # The cart 
    rod2, = plt.plot([Z2[0, 0], X2[0]], [0, Y2[0]], color="black")   # The massless pendulum rod of length l
    bead2, = plt.plot(X2[0], Y2[0], 'o', color="black", ms=4)        # The pendulum bead
    text2 = plt.text(xMax2-0.1, 0.5*yDomain2 - 0.1, r'$t =  %.2f$s'%(timeValues[0]), # Text box with elapsed time
            {'color': 'k', 'fontsize': 10, 'ha': 'center', 'va': 'center',
            'bbox': dict(boxstyle='round', fc='w', ec='k', pad=0.2)})

    plt.xlabel('x, m')
    plt.ylabel('y, m')
    plt.title("Linear Quadratic Regulator")
    
    plt.subplots_adjust(hspace=0.3)
    
    # Calculate the number of frames
    FPS = 30
    framesNum = int(FPS*duration)
    dataPointsPerFrame = int(n/framesNum)

    # Animation function. This is called sequentially
    def animate(j):
        time = j*dataPointsPerFrame
        # Pendulum 1
        tail.set_xdata(X[:time])
        tail.set_ydata(Y[:time])
        cart.set_xdata(cartX(Z[0, time])) 
        cart.set_ydata(cartY)
        rod.set_xdata([Z[0, time], X[time]])
        rod.set_ydata([0, Y[time]])
        bead.set_xdata(X[time])
        bead.set_ydata(Y[time])
        text.set_text(r'$t =  %.2f$s'%(timeValues[time]))
        
        # Pendulum 2
        tail2.set_xdata(X2[:time])
        tail2.set_ydata(Y2[:time])
        cart2.set_xdata(cartX(Z2[0, time])) 
        cart2.set_ydata(cartY2)
        rod2.set_xdata([Z2[0, time], X2[time]])
        rod2.set_ydata([0, Y2[time]])
        bead2.set_xdata(X2[time])
        bead2.set_ydata(Y2[time])
        text2.set_text(r'$t =  %.2f$s'%(timeValues[time]))

    # Create animation
    anim = animation.FuncAnimation(fig, animate, frames=framesNum)

    # Save animation.
    # If this don't work for you, try using another writer 
    # (ffmpeg, mencoder, imagemagick), or another file extension
    # (.mp4, .gif, .ogg, .ogv, .avi etc.). Make sure that you
    # have the codec and the writer installed on your system.
    anim.save(name, writer='pillow', fps=FPS)

    # Close plot
    plt.close(anim._fig)

    # Display the animation
    with open(name, 'rb') as file:
        display(Image(file.read()))

In [ ]:
def generateAnimationWithPhaseSpace(Z, name):
    """ This function generates an animation consisting of a plot of the pendulum
    and phase-space plot of the two generalised coordinates x and theta, as a
    functino of time.
    Arguments:
            Z      4xN array. State vector component-values at each timestep
            name   string. Name of the animation file.
    """
    
    X = Z[0] + l*np.sin(Z[1])                           # x-coordinates of bead
    Y = l*np.cos(Z[1])                                  # y-coordinates of bead
    cartY = np.array([l/16, l/16, -l/16, -l/16, l/16])  # y-coordinates of cart corners

    fig = plt.figure(figsize=(figSizeX, figSizeY), dpi=200)
    # Axis limitations
    plt.subplot(2, 2, (1, 3))
    xMin = np.min(X) - l/2 
    xMax = np.max(X) + l/2
    # Adjust the y-limitations such that a circle looks like a circle.
    # i.e. no scaling.
    xDomain = xMax - xMin
    yDomain = xDomain*2*figSizeY/figSizeX
    plt.xlim(xMin, xMax)
    plt.ylim(-0.5*yDomain, 0.5*yDomain)

    # Defining the different elements in the animation
    surface, = plt.plot([xMin, xMax], [-l/16, -l/16], color='black', linewidth=1)    # The surface
    tail, = plt.plot(X[0], Y[0], '--', color="blue")            # Previous position of the pendulum bead
    cart, = plt.plot(cartX(Z[0, 0]), cartY, color="red")        # The cart 
    rod, = plt.plot([Z[0, 0], X[0]], [0, Y[0]], color="black")  # The massless pendulum rod of length l
    bead, = plt.plot(X[0], Y[0], 'o', color="black", ms=4)      # The pendulum bead

    plt.xlabel('x')
    plt.ylabel('y')
    plt.title("Inverted pendulum on cart")
    
    # Phase-space plot of the rod angle theta
    plt.subplot(222)
    plt.title("Rod angle in phase-space")
    anglePath, = plt.plot(Z[1], Z[3])
    angleFinal, = plt.plot(Z[1, -1], Z[3, -1], 'o', label="Final point")
    angleCurrent, = plt.plot(Z[1, 0], Z[3, 0], '*', label="Current point")
    plt.xlabel(r'$\theta$ [rad]')
    plt.ylabel(r'$\dot{\theta}$ [rad/s]')
    plt.legend(loc='best')
    plt.grid(linestyle=":")
    
    # Phase-space plot of the cart position x.
    plt.subplot(224)
    plt.title("Cart position in phase-space")
    cartPath, = plt.plot(Z[0], Z[2])
    cartFinal, = plt.plot(Z[0, -1], Z[2, -1], 'o', label="Final point")
    cartCurrent, = plt.plot(Z[0, 0], Z[2, 0], '*', label="Current point")
    plt.xlabel(r'$x$ [m]')
    plt.ylabel(r'$\dot{x}$ [m/s]')
    plt.legend(loc='best')
    plt.grid(linestyle=":")    
    
    # Calculate the number of frames
    FPS = 30
    framesNum = int(FPS*duration)
    dataPointsPerFrame = int(n/framesNum)

    # Animation function. This is called sequentially
    def animate(j):
        time = j*dataPointsPerFrame
        
        # Updating each element every time 
        tail.set_xdata(X[:time])
        tail.set_ydata(Y[:time])
        cart.set_xdata(cartX(Z[0, time])) 
        cart.set_ydata(cartY)
        rod.set_xdata([Z[0, time], X[time]])
        rod.set_ydata([0, Y[time]])
        bead.set_xdata(X[time])
        bead.set_ydata(Y[time])
        
        angleCurrent.set_xdata(Z[1, time])
        angleCurrent.set_ydata(Z[3, time])
        anglePath.set_xdata(Z[1, :time])
        anglePath.set_ydata(Z[3, :time])
        cartCurrent.set_xdata(Z[0, time])
        cartCurrent.set_ydata(Z[2, time])
        cartPath.set_xdata(Z[0, :time])
        cartPath.set_ydata(Z[2, :time])

    # Create animation
    anim = animation.FuncAnimation(fig, animate, frames=framesNum)

    # Save animation.
    # If this don't work for you, try using another writer 
    # (ffmpeg, mencoder, imagemagick), or another file extension
    # (.mp4, .gif, .ogg, .ogv, .avi etc.). Make sure that you
    # have the codec and the writer installed on your system.
    anim.save(name, writer='pillow', fps=FPS)

    # Close plot
    plt.close(anim._fig)

    # Display the animation
    with open(name, 'rb') as file:
        display(Image(file.read()))